In [ ]:
import pandas as pd
import numpy as np
import gc
from scipy import interpolate
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.model_selection import cross_val_score
from lightgbm.sklearn import LGBMClassifier
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from datetime import datetime, timedelta
from sklearn.metrics import plot_roc_curve,roc_curve,auc,roc_auc_score
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [ ]:
train_x = pd.read_csv('/Users/yangxianjie/测试项目/AI/cube_data/train_x_12_feature_engineering.csv')
train_y = pd.read_csv('/Users/yangxianjie/测试项目/AI/cube_data/train_y_12_feature_engineering.csv')
usecol_x = [col for col in train_x.columns if col not in ['Unnamed: 0']]
train_x= train_x[usecol_x]
usecol_y = [col for col in train_y.columns if col not in ['Unnamed: 0']]
train_y = train_y[usecol_y]

val_x = pd.read_csv('/Users/yangxianjie/测试项目/AI/cube_data/valid_x_12_feature_engineering.csv')
val_y = pd.read_csv('/Users/yangxianjie/测试项目/AI/cube_data/valid_y_12_feature_engineering.csv')
usecol_val_x = [col for col in val_x.columns if col not in ['Unnamed: 0']]
val_x= val_x[usecol_val_x]
usecol_val_y = [col for col in val_y.columns if col not in ['Unnamed: 0']]
val_y = val_y[usecol_val_y]

input_x, _, input_y, _ = train_test_split(train_x, train_y, train_size=0.4, random_state=2021, stratify=train_y)

# Decision Tree
dtc = tree.DecisionTreeClassifier(
    splitter = 'best',
    min_samples_leaf = 1,
    min_samples_split = 2,
    max_depth = 45,
    max_features = 3,
    random_state = 2021
)

# SVM
scl = svm.SVC(C = 1, 
              kernel = 'rbf', 
              gamma = 1, 
              decision_function_shape = 'ovr')

# Random Forest
rfc = RandomForestClassifier(n_estimators = 94,
                             random_state = 2021
)

# AdaBoost
weakClassifier = tree.DecisionTreeClassifier(
    splitter = 'best',
    min_samples_leaf = 1,
    min_samples_split = 2,
    max_depth = 45,
    max_features = 3,
    random_state = 2021
)

abc = AdaBoostClassifier(base_estimator = weakClassifier, 
                         n_estimators = 91, 
                         learning_rate = 0.1)

for i in [0.2, 0.4, 0.6, 0.8]:
    split_train_x, split_valid_x, split_train_y, split_valid_y = train_test_split(input_x, input_y, train_size = i, random_state = 2021, stratify = input_y)

    dtc.fit(split_train_x, split_train_y)
    scl.fit(split_train_x, split_train_y)
    rfc.fit(split_train_x, split_train_y)
    abc.fit(split_train_x, split_train_y)

    # Decision Tree
    output_pred_dtc = dtc.predict(split_valid_x)
    
    split_score_f1_dtc = f1_score(split_valid_y, output_pred_dtc)
    print('train_score_f1_dtc=', split_score_f1_dtc)

    print(classification_report(split_valid_y, output_pred_dtc))
    print(confusion_matrix(split_valid_y, output_pred_dtc))
    print('==============================')
    
    # SVM
    output_pred_svm = scl.predict(split_valid_x)
    
    split_score_f1_svm = f1_score(split_valid_y, output_pred_svm)
    print('train_score_f1_svm=', split_score_f1_svm)

    print(classification_report(split_valid_y, output_pred_svm))
    print(confusion_matrix(split_valid_y, output_pred_svm))
    print('==============================')
    
    #Random Forest
    output_pred_rfc = rfc.predict(split_valid_x)
    
    split_score_f1_rfc = f1_score(split_valid_y, output_pred_rfc)
    print('train_score_f1_rfc=', split_score_f1_rfc)

    print(classification_report(split_valid_y, output_pred_rfc))
    print(confusion_matrix(split_valid_y, output_pred_rfc))
    print('==============================')
    
    # AdaBoost
    output_pred_abc = abc.predict(split_valid_x)
    
    split_score_f1_abc = f1_score(split_valid_y, output_pred_abc)
    print('train_score_f1_rfc=', split_score_f1_abc)

    print(classification_report(split_valid_y, output_pred_abc))
    print(confusion_matrix(split_valid_y, output_pred_abc))
    print('************************************************')
    